# Session 17 - Visualisations Interactives avec Plotly

## 🎯 Objectifs
- Créer 3 visualisations interactives professionnelles
- Raconter une histoire avec les données du Titanic
- Exporter en HTML pour partage

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print('✓ Plotly importé')
print(f'Version Plotly : {px.__version__ if hasattr(px, "__version__") else "N/A"}')

In [ ]:
# Charger les données
df = pd.read_csv('../data/titanic_clean.csv')

print(f'Dataset : {df.shape[0]} lignes x {df.shape[1]} colonnes')
df.head()

## Visualisation 1 : Taux de survie par classe et sexe (Heatmap interactive)

In [ ]:
# Préparer les données
survival_matrix = df.groupby(['Pclass', 'Sex'])['Survived'].mean().unstack() * 100

# Créer la heatmap
fig = go.Figure(data=go.Heatmap(
    z=survival_matrix.values,
    x=['Homme', 'Femme'],
    y=['1ère classe', '2ème classe', '3ème classe'],
    text=survival_matrix.values.round(1),
    texttemplate='%{text}%',
    textfont={'size': 16, 'color': 'white'},
    colorscale='RdYlGn',
    colorbar=dict(title='Taux de<br>survie (%)')
))

fig.update_layout(
    title={
        'text': 'Les femmes de 1ère classe avaient ~97% de chances de survivre<br><sub>Taux de survie par classe et sexe</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#2C3E50'}
    },
    xaxis_title='Sexe',
    yaxis_title='Classe',
    width=800,
    height=500,
    font=dict(size=12, family='Arial'),
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA'
)

fig.show()

# Sauvegarder
fig.write_html('viz1_survival_heatmap.html')
print('✓ Sauvegardé : viz1_survival_heatmap.html')

## Visualisation 2 : Distribution d'âge avec survie (Violin plot interactif)

In [ ]:
# Créer le violin plot
fig = px.violin(df, y='Age', x='Pclass', color='Survived',
                box=True,  # Ajouter boxplot
                points='outliers',  # Montrer les outliers
                color_discrete_map={0: '#E74C3C', 1: '#2ECC71'},
                labels={'Survived': 'Survie', 'Pclass': 'Classe', 'Age': 'Âge (années)'})

fig.update_layout(
    title={
        'text': 'Les enfants (< 10 ans) de toutes classes avaient un meilleur taux de survie<br><sub>Distribution de l\'âge par classe et survie</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#2C3E50'}
    },
    xaxis_title='Classe du billet',
    yaxis_title='Âge (années)',
    legend_title_text='Survie',
    width=1000,
    height=600,
    font=dict(size=12, family='Arial'),
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    hovermode='closest'
)

# Renommer les étiquettes de la légende
fig.for_each_trace(lambda t: t.update(name='Non-survivant' if t.name == '0' else 'Survivant'))

# Ajouter une ligne de référence
fig.add_hline(y=10, line_dash='dash', line_color='blue', opacity=0.5,
              annotation_text='Âge 10 ans', annotation_position='right')

fig.show()

# Sauvegarder
fig.write_html('viz2_age_distribution.html')
print('✓ Sauvegardé : viz2_age_distribution.html')

## Visualisation 3 : Analyse multidimensionnelle (Bubble chart 3D)

In [ ]:
# Préparer les données agrégées pour plus de clarté
agg_df = df.groupby(['Pclass', 'Sex', 'Survived']).agg({
    'Age': 'mean',
    'Fare': 'mean',
    'PassengerId': 'count'  # Utiliser PassengerId ou créer une colonne de comptage
}).reset_index()

# Renommer pour clarté
agg_df.columns = ['Pclass', 'Sex', 'Survived', 'Age_mean', 'Fare_mean', 'Count']
agg_df['Survived_label'] = agg_df['Survived'].map({0: 'Non-survivant', 1: 'Survivant'})
agg_df['Sex_label'] = agg_df['Sex'].map({0: 'Homme', 1: 'Femme'})

# Créer le bubble chart
fig = px.scatter(agg_df, 
                 x='Age_mean', 
                 y='Fare_mean',
                 size='Count',  # Taille = nombre de passagers
                 color='Survived_label',
                 facet_col='Pclass',
                 hover_data={'Sex_label': True, 'Count': True, 
                            'Age_mean': ':.1f', 'Fare_mean': ':.2f',
                            'Survived_label': False, 'Pclass': False},
                 color_discrete_map={'Non-survivant': '#E74C3C', 'Survivant': '#2ECC71'},
                 labels={'Age_mean': 'Âge moyen', 'Fare_mean': 'Prix moyen (£)',
                        'Count': 'Nombre', 'Survived_label': 'Survie'})

fig.update_layout(
    title={
        'text': 'Profils de passagers : âge, prix et survie par classe<br><sub>Taille des bulles = nombre de passagers dans chaque groupe</sub>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#2C3E50'}
    },
    width=1400,
    height=500,
    font=dict(size=11, family='Arial'),
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA',
    showlegend=True,
    legend=dict(title='Survie', orientation='v', x=1.02, y=0.5)
)

# Mettre à jour les titres des facettes
fig.for_each_annotation(lambda a: a.update(text=a.text.replace('Pclass=', 'Classe ')))

fig.show()

# Sauvegarder
fig.write_html('viz3_multidimensional_analysis.html')
print('✓ Sauvegardé : viz3_multidimensional_analysis.html')

## Dashboard complet : Vue d'ensemble

In [ ]:
# Créer un dashboard avec 4 visualisations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Taux de survie par classe',
                    'Répartition par sexe et survie',
                    'Distribution des prix',
                    'Taille de famille vs survie'),
    specs=[[{'type': 'bar'}, {'type': 'pie'}],
           [{'type': 'box'}, {'type': 'scatter'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Bar chart - Survie par classe
survival_class = df.groupby('Pclass')['Survived'].mean()
fig.add_trace(
    go.Bar(x=['1ère', '2ème', '3ème'], y=survival_class.values*100,
           marker_color=['#2ECC71', '#F39C12', '#E74C3C'],
           text=[f'{v*100:.1f}%' for v in survival_class.values],
           textposition='outside',
           name='Survie'),
    row=1, col=1
)

# 2. Pie chart - Sexe et survie
survival_sex = df.groupby(['Sex', 'Survived']).size().reset_index(name='count')
survival_sex['label'] = survival_sex.apply(
    lambda x: f"{'Homme' if x['Sex']==0 else 'Femme'} - {'Survivant' if x['Survived']==1 else 'Non-survivant'}", 
    axis=1
)
fig.add_trace(
    go.Pie(labels=survival_sex['label'], values=survival_sex['count'],
           marker=dict(colors=['#E74C3C', '#2ECC71', '#C0392B', '#27AE60'])),
    row=1, col=2
)

# 3. Box plot - Prix par classe
for pclass in [1, 2, 3]:
    fig.add_trace(
        go.Box(y=df[df['Pclass']==pclass]['Fare'], name=f'Classe {pclass}',
               marker_color=['#2ECC71', '#F39C12', '#E74C3C'][pclass-1]),
        row=2, col=1
    )

# 4. Scatter - Famille vs survie
family_survival = df.groupby('FamilySize').agg({'Survived': 'mean', 'PassengerId': 'count'}).reset_index()
fig.add_trace(
    go.Scatter(x=family_survival['FamilySize'], 
               y=family_survival['Survived']*100,
               mode='lines+markers',
               marker=dict(size=family_survival['PassengerId']/10, color='#3498DB'),
               line=dict(color='#3498DB', width=2),
               name='Taux'),
    row=2, col=2
)

# Layout global
fig.update_layout(
    title_text='Dashboard Titanic - Vue d\'ensemble interactive',
    title_x=0.5,
    title_font=dict(size=20, color='#2C3E50'),
    showlegend=False,
    height=800,
    width=1400,
    plot_bgcolor='white',
    paper_bgcolor='#F8F9FA'
)

# Axes
fig.update_xaxes(title_text='Classe', row=1, col=1)
fig.update_yaxes(title_text='Taux de survie (%)', row=1, col=1)
fig.update_yaxes(title_text='Prix (£)', row=2, col=1)
fig.update_xaxes(title_text='Taille de famille', row=2, col=2)
fig.update_yaxes(title_text='Taux de survie (%)', row=2, col=2)

fig.show()

# Sauvegarder
fig.write_html('dashboard_titanic.html')
print('✓ Sauvegardé : dashboard_titanic.html')

## Récapitulatif des insights

### 🎯 Insights clés découverts :

1. **Classe et sexe** : Facteurs déterminants
   - Femmes de 1ère classe : ~97% de survie
   - Hommes de 3ème classe : ~13% de survie
   - Différence de survie x7 entre ces groupes

2. **Âge** : Priorité aux enfants
   - Enfants < 10 ans : Taux de survie élevé dans toutes classes
   - Principe 'femmes et enfants d'abord' appliqué
   - Âge moyen des survivants légèrement inférieur

3. **Taille de famille** : Effet non-linéaire
   - Familles de 2-4 personnes : Meilleur taux (~70%)
   - Seuls ou grandes familles : Taux plus faible (~30-40%)
   - Entraide vs coordination difficile

4. **Prix** : Reflet de la classe
   - Corrélation forte entre prix et survie
   - Prix élevé = meilleur emplacement cabine
   - Accès plus rapide aux canots

### 📊 Fichiers créés :

- `viz1_survival_heatmap.html` - Heatmap interactive survie par classe/sexe
- `viz2_age_distribution.html` - Violin plot âge et survie
- `viz3_multidimensional_analysis.html` - Bubble chart multidimensionnel
- `dashboard_titanic.html` - Dashboard complet 4 visualisations

Tous les fichiers HTML sont **interactifs** et peuvent être partagés directement !